In [1]:
import os, glob
from typing import List
os.chdir("/home/rygielpt/lddmm")

import pyvista as pv
import pandas as pd
from src.lddmm.registrator import LDDMMRegistrator

[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


In [2]:
class LongitudinalSample:
    def __init__(self, root_dir: str, name: str):
        self.root_dir = root_dir
        self.name = name
        self.data = self._assemble_dataframe()

        self.registrator = None

    def _assemble_dataframe(self):
        def _parse_date(filepath: str):
            raw_date = filepath.split("/")[-2].split("_")[1]
            return pd.to_datetime(f"{raw_date[:4]}-{raw_date[4:6]}-{raw_date[6:8]}")

        data = pd.DataFrame(
            [
                (_parse_date(file), file) 
                for file in sorted(glob.glob(os.path.join(self.root_dir, f"{self.name}*", "full.vtp")))
            ],
            columns=["date", "path"]
        )
        data["days"] = (data["date"] - data.iloc[0]["date"]).dt.days
        return data
    
    def fit_LMMDD(
        self, 
        source_t: int = 0, 
        target_ts: List[int] = [-1], 
        centralize: bool = True,
        optimizer_iters: int = 10, 
        ode_steps: int = 10, 
        sigma: float = 20,
        device: str = "cpu"
    ):
        # Load source mesh
        source_days, source_file = self.data.iloc[source_t][["days", "path"]]
        source_mesh = pv.read(source_file)

        # Load target meshes
        target_dict = {
            self.data.iloc[i]["days"] - source_days: pv.read(self.data.iloc[i]["path"]) for i in target_ts
        }
        target_dict = {
            round(ode_steps * t / max(target_dict.keys())): mesh for t, mesh in target_dict.items()
        }

        # Rigid centralization
        if centralize:
            source_mesh.points -= source_mesh.points.mean(axis=0)
            
            for k in target_dict.keys():
                target_dict[k].points -= target_dict[k].points.mean(axis=0)

        # Optimize
        self.registrator = LDDMMRegistrator(source_mesh, target_dict, sigma=sigma, device=device)
        self.registrator.optimize(optimizer_iters, ode_steps)

    def infer_LMMDD(self, output_dir: str, steps: int = 15, deltat: int = 1.0):
        self.registrator.export_shoot(output_dir, nt=steps, deltat=deltat)

In [3]:
root_dir = "/home/rygielpt/datasets/BAEK/meshes"
patient_id = "P20"

sample = LongitudinalSample(root_dir, patient_id)
sample.data

,date,path,days
0,2008-02-12,/home/rygielpt/datasets/BAEK/meshes/P20_200802...,0
1,2008-07-28,/home/rygielpt/datasets/BAEK/meshes/P20_200807...,167
2,2009-01-19,/home/rygielpt/datasets/BAEK/meshes/P20_200901...,342
3,2010-01-21,/home/rygielpt/datasets/BAEK/meshes/P20_201001...,709
4,2010-04-12,/home/rygielpt/datasets/BAEK/meshes/P20_201004...,790
5,2011-03-10,/home/rygielpt/datasets/BAEK/meshes/P20_201103...,1122


In [4]:
sample.fit_LMMDD(0, [-1], device="cuda")

[KeOps] Generating code for Sum_Reduction reduction (with parameters 0) of formula ((c|d)**2*e)*Exp(-(Sum((a-b)**2)*f)) with a=Var(0,3,0), b=Var(1,3,1), c=Var(2,3,0), d=Var(3,3,1), e=Var(4,1,1), f=Var(5,1,2) ... OK
[pyKeOps] Compiling pykeops cpp fa37694969 module ... OK


: 